# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics1 import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\retail_train.csv')
item_features = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\product.csv')
user_features = pd.read_csv('C:\\Users\\mmingalov\\geekbrains-recommend-systems\\hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, N=200)

[899624,
 871756,
 1044078,
 1106523,
 832678,
 5569230,
 844179,
 883932,
 965766,
 1046545,
 1116376,
 879504,
 1004906,
 999714,
 865456,
 963727,
 925862,
 944534,
 1138467,
 863447,
 952563,
 1003616,
 823990,
 870547,
 854852,
 1092937,
 892503,
 935578,
 1134678,
 7441873,
 8090521,
 1025535,
 903609,
 947858,
 1081177,
 841220,
 1068719,
 993638,
 1100972,
 5568378,
 824180,
 5585510,
 952163,
 990797,
 8065410,
 8090537,
 9836195,
 998556,
 937292,
 940090,
 837304,
 1036347,
 835530,
 1037863,
 1123146,
 1097458,
 1051323,
 832442,
 12731432,
 828106,
 9836106,
 999779,
 880150,
 965267,
 1004390,
 1017369,
 839818,
 1081262,
 1029743,
 910032,
 896613,
 850925,
 1042438,
 986912,
 1021324,
 6534480,
 12731714,
 826666,
 1131344,
 902172,
 830304,
 868909,
 1012587,
 999858,
 1000753,
 1002850,
 12302069,
 12301839,
 1042942,
 845078,
 865528,
 835300,
 1096635,
 9835606,
 1035207,
 913278,
 9526410,
 888543,
 822178,
 9835223,
 850102,
 10149640,
 1042907,
 1134296,
 829138,

In [6]:
recommender.get_own_recommendations(2375, N=200)

[948640,
 918046,
 847962,
 907099,
 873980,
 884694,
 10285454,
 1107760,
 7169090,
 979674,
 10308345,
 1069531,
 974766,
 1015474,
 950935,
 847066,
 1102207,
 1020770,
 9521787,
 974265,
 940996,
 8019845,
 5567194,
 12811490,
 1003616,
 973181,
 890719,
 982955,
 9677152,
 998519,
 1072685,
 1131382,
 1021715,
 12263119,
 960791,
 7441873,
 986021,
 956666,
 1038692,
 9677748,
 9297223,
 927030,
 12757653,
 1046919,
 6391532,
 989069,
 1068451,
 951954,
 835300,
 937343,
 1047249,
 13876348,
 1061732,
 981601,
 1121028,
 1087547,
 828393,
 996269,
 951951,
 1036093,
 1023815,
 5570408,
 827667,
 1082454,
 1006878,
 5570048,
 841309,
 1078652,
 1115553,
 1056492,
 1138467,
 1004945,
 947858,
 1092885,
 1121694,
 938138,
 8019916,
 827919,
 984315,
 10341855,
 883932,
 8291322,
 1096794,
 1028938,
 1087618,
 8020166,
 1082185,
 866871,
 930666,
 825994,
 910151,
 823990,
 848029,
 896613,
 12301839,
 1117219,
 1135258,
 869868,
 1046545,
 899624,
 6442594,
 1137775,
 825343,
 104290

In [7]:
recommender.get_similar_items_recommendation(2375, N=200)

[1046545,
 1044078,
 1042907,
 1115576,
 1133312,
 1051323,
 1005274,
 896613,
 885863,
 1025535,
 863447,
 878996,
 918846,
 871611,
 865528,
 919681,
 828106,
 1018740,
 1127831,
 837569,
 1070702,
 851394,
 858373,
 12301100,
 899624,
 968363,
 9297571,
 823721,
 985999,
 1037417,
 9835903,
 1127025,
 1000753,
 992826,
 6979248,
 1045069,
 9420140,
 825343,
 1090931,
 912704,
 990762,
 916122,
 869322,
 912301,
 9655679,
 862538,
 947201,
 1048299,
 8065410,
 1056005,
 954651,
 960732,
 8020234,
 898068,
 12755327,
 1069312,
 1043751,
 828106,
 883068,
 1105488,
 990797,
 825343,
 1120559,
 1038998,
 899459,
 1077133,
 8090537,
 882247,
 8090521,
 948650,
 13876348,
 12262778,
 969977,
 1042083,
 957411,
 969945,
 9677543,
 5592261,
 889105,
 1005576,
 937292,
 12301109,
 999858,
 9220882,
 1014116,
 913278,
 9677149,
 1139142,
 1046919,
 7441210,
 944141,
 12757425,
 889759,
 1119075,
 1048507,
 954651,
 10150194,
 937292,
 885863,
 9419444,
 10254193,
 959316,
 7024871,
 902094,
 

In [8]:
recommender.get_similar_users_recommendation(2375, N=200)

[10457044,
 896757,
 974265,
 1057168,
 1101502,
 1107760,
 1037135,
 1092588,
 9553048,
 837969,
 837495,
 10198378,
 894360,
 1124971,
 1030093,
 830686,
 994577,
 926006,
 1015539,
 965956,
 1060292,
 12262832,
 12427353,
 1117219,
 12352054,
 1010308,
 847434,
 874563,
 12523928,
 1118946,
 873715,
 1038745,
 873324,
 820612,
 894360,
 7443137,
 963686,
 949257,
 921438,
 917033,
 892728,
 871514,
 1129805,
 1071196,
 9245108,
 12523928,
 9553382,
 873044,
 8066803,
 9926758,
 939860,
 1114653,
 914245,
 1026945,
 916990,
 839605,
 1055403,
 945909,
 1114811,
 897270,
 977780,
 1118120,
 879699,
 977927,
 865757,
 8090956,
 8090542,
 882826,
 871722,
 928932,
 956125,
 833151,
 10198378,
 1081533,
 857538,
 875392,
 948239,
 939681,
 979674,
 1096573,
 937736,
 852015,
 854133,
 921406,
 9392700,
 1057168,
 1126045,
 1134296,
 873324,
 1069256,
 1036699,
 10284929,
 8090533,
 951834,
 6772833,
 970160,
 859987,
 1131625,
 6632283,
 835578,
 973016,
 965772,
 825317,
 918638,
 10571

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 200 кандидатов (k=200)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
# your_code

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# your_code

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - money precision@5. Порог для уcпешной сдачи проекта money precision@5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- **2 новых товара** (юзер никогда не покупал)
- **1 дорогой товар, > 7 долларов**
- **Все товары из разных категорий** (категория - department)  
- **Стоимость каждого рекомендованного товара > 1 доллара**  

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)